[fractalego](https://huggingface.co/fractalego/fact-checking?text=My+name+is+Julien+and+I+like+to)

In [ ]:
pip install transformers

In [ ]:
pip install fact_checking

In [ ]:
import json
from tqdm import tqdm


In [ ]:
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
)

from fact_checking import FactChecker
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
fact_checking_model = GPT2LMHeadModel.from_pretrained('fractalego/fact-checking')
fact_checker = FactChecker(fact_checking_model, tokenizer)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/861 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
def get_prediction(filename):
  print(f'Processing file: {filename}')
  with open(filename) as f:
    data = json.load(f)


  results = []
  results_both_same = 0

  tp, tn, fp, fn = 0, 0, 0, 0
  for spouse_data in tqdm(data):
    person_one, person_two = spouse_data[0]
    claim_1 = f'{person_one} is married to {person_two}.' 
    claim_2 = f'{person_two} is married to {person_one}.' 

    correct = spouse_data[2]
    evidence = spouse_data[3]
    result_1 = fact_checker.validate(evidence, claim_1)
    results.append((claim_1, evidence, correct, result_1))
    tp += 1 if (result_1 == correct == True) else 0
    tn += 1 if (result_1 == correct == False) else 0
    fp += 1 if (result_1 == True and correct == False) else 0
    fn += 1 if (result_1 == False and correct == True) else 0

    result_2 = fact_checker.validate(evidence, claim_2)
    results.append((claim_2, evidence, correct, result_2))
    tp += 1 if (result_2 == correct == True) else 0
    tn += 1 if (result_2 == correct == False) else 0
    fp += 1 if (result_2 == True and correct == False) else 0
    fn += 1 if (result_2 == False and correct == True) else 0
    
    results_both_same += 1 if (result_1 == result_2) else 0

  
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)

  print('Done')
  print(f'tp={tp},fp={fp},tn={tn},fn={fn}')
  print(f'Precision: {precision}')
  print(f'Recall: {recall}')
  print(f'F1 score: {(2*precision*recall)/(precision + recall)}')
  print(f'Count of both same result: {results_both_same}')

  return results

In [ ]:
results = get_prediction('./spouse_fact_validation_gt.json')

Processing file: ./spouse_fact_validation_gt.json


100%|██████████| 100/100 [00:05<00:00, 16.84it/s]

Done
tp=103,fp=58,tn=30,fn=9
Precision: 0.639751552795031
Recall: 0.9196428571428571
F1 score: 0.7545787545787545
Count of both same result: 77


In [ ]:
import textwrap

In [ ]:
# FP
selection = [x for x in results if (x[2] == False and x[3] == True)] 
# FN
# selection = [x for x in results if (x[2] == True and x[3] == False)] 
# TP
# selection = [x for x in results if (x[2] == x[3] == True)] 
# TP
# selection = [x for x in results if (x[2] == x[3] == False)] 

for x in selection:
  print('-'*50)
  print(f'Claim: {x[0]}')
  print('-'*5)
  print(textwrap.fill(f'Evidence: {x[1]}', 80))
  print('-'*5)
  print(f'Correct: {x[2]}')
  print('-'*5)
  print(f'Predicted: {x[3]}')

--------------------------------------------------
Claim: Deborah Nadoolman Landis is the spouse of John Landis.
-----
Evidence: "Sheila Landis" more than 30 years, Landis has never fallen into a
rut. Her work continues to exude vitality, excitement, diversity, and
originality"". Not content to simply sing and scat in the jazz vocal tradition,
Landis takes the lead from vocal innovators Al Jarreau and Bobby McFerrin in
expanding her vocal palette to include uncanny imitations of the harmonica,
trumpet, flute, synthesizer, string bass and even drums. Sheila Landis Sheila
Catherine Landis (born September 10, 1953, Detroit, Michigan, United States) is
an American jazz, R&B vocalist and songwriter. In 1981, Landis formed her own
label, SheLan Records, initially releasing five albums of
-----
Correct: False
-----
Predicted: True
--------------------------------------------------
Claim: John Landis is the spouse of Deborah Nadoolman Landis.
-----
Evidence: "Sheila Landis" more than 30 years,